In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import sqlite3

In [2]:
def get_dish_urls():
    """Retrieve URLS for dishes under chicken, pork, and beef categories"""
    # Limit main ingredients to three types of meat
    meats = ['chicken', 'pork', 'beef']

    # Prepare dish URLs holder
    dish_urls = {}

    # Loop through each meat types
    for meat in meats:

        # Get HTML for the category page
        response = requests.get(
            f'https://panlasangpinoy.com/{meat}-recipes/'
        )
        soup = BeautifulSoup(response.text)

        # Get number of pages
        pagination = soup.select_one(
            "div[class='archive-pagination pagination']"
        )
        last_page = pagination.select_one(
            "li[class='pagination-omission'] + li"
        )
        num_pages = int(last_page.select_one("a").contents[1])

        # Loop through each pages per meat type
        for n in np.arange(1, num_pages+1):
            response = requests.get(
                f'https://panlasangpinoy.com/{meat}-recipes/page/{n}/'
            )
            soup = BeautifulSoup(response.text)

            # Store dish URL
            for dish in soup.select("a[class='entry-title-link']"):
                dish_name = dish.contents[0]
                dish_urls[dish_name] = dish['href']
                
    print("Extraction of URLs done. Total number of dishes:", len(dish_urls))

    return dish_urls

def get_ingredients(dish_urls):
    """Get ingredients for all dishes"""
    dish_dict = {}

    for key, url in dish_urls.items():
        print("Getting details for: ",key)
        response = requests.get(url)
        soup = BeautifulSoup(response.text)

        dish_details = {}

        try:
            dish_details['course'] = soup.select_one(
                "span[class='wprm-recipe-course "
                "wprm-block-text-normal']"
            ).contents[0]
        except AttributeError:
            dish_details['course'] = np.nan

        try:
            dish_details['cuisine'] = soup.select_one(
                "span[class='wprm-recipe-cuisine "
                "wprm-block-text-normal']"
            ).contents[0]
        except AttributeError:
            dish_details['cusine'] = np.nan
        
        try:
            dish_details['prep_time'] = soup.select_one(
                "span[class='wprm-recipe-details-label wprm-block-text-bold "
                "wprm-recipe-time-label wprm-recipe-prep-time-label'] + span"
            ).text
        except AttributeError:
            dish_details['prep_time'] = np.nan

        try:
            dish_details['cook_time'] = soup.select_one(
                "span[class='wprm-recipe-details-label wprm-block-text-bold "
                "wprm-recipe-time-label wprm-recipe-cook-time-label'] + span"
            ).text
        except AttributeError:
            dish_details['cook_time'] = np.nan
            
        try:
            dish_details['total_time'] = soup.select_one(
                "span[class='wprm-recipe-details-label wprm-block-text-bold "
                "wprm-recipe-time-label wprm-recipe-total-time-label'] + span"
            ).text
        except AttributeError:
            dish_details['total_time'] = np.nan

        try:
            servings = soup.select_one(
                "span[class='wprm-recipe-details-label wprm-block-text-bold "
                "wprm-recipe-servings-label']"
            ).next_sibling.text
            dish_details['servings'] = int(re.findall('\d+', servings)[0])
        except AttributeError:
            dish_details['servings'] = np.nan

        try:
            dish_details['calories'] = soup.select_one(
                "span[class='wprm-recipe-nutrition-with-unit']"
            ).text
        except AttributeError:
            dish_details['calories'] = np.nan

        for ing in soup.select("li[class='wprm-recipe-ingredient']"):
            ing_name = ing.select_one(
                "span[class='wprm-recipe-ingredient-name']"
            ).text
            try:
                str_amount = ing.select_one(
                    "span[class='wprm-recipe-ingredient-amount']"
                ).text
                str_unit = ing.select_one(
                    "span[class='wprm-recipe-ingredient-unit']"
                ).text
                dish_details[ing_name] = get_weight(str_amount, str_unit)
            except AttributeError:
                dish_details[ing_name] = get_weight(ing_name, ing_name)
            
        dish_dict[key] = dish_details
        
    return dish_dict

def get_weight(str_amount, str_weight):
    """Transform different measurements into base units"""
    amount = 0
    div_weight = 1
    
    # Set dictionary of vulgar fraction unicode characters
    vul_frac = {
        '¼':1/4,
        '½':.5,
        '¾':3/4,
        '⅐':1/7,
        '⅑':1/9,
        '⅒':.1,
        '⅓':1/3,
        '⅔':2/3,
        '⅕':.2,
        '⅖':.4,
        '⅗':.6,
        '⅘':.8,
        '⅙':1/6,
        '⅚':5/6,
        '⅛':1/8,
        '⅜':3/8,
        '⅝':5/8,
        '⅞':7/8
    }

    # Check for vulgar fraction characters
    for vul_char, frac in vul_frac.items():
        if str_amount.find(vul_char) != -1:
            amount += frac
            break
    
    # Check for typed fractions
    frac_list = re.findall(r'(\d+)/(\d+)', str_amount)
    if frac_list != []:
        num_frac, den_frac = frac_list[0]
        amount += int(num_frac) / int(den_frac)
    
    # Check for whole number
    whole_num_list = re.findall(r'(?<!/)(\d+)(?!/)', str_amount)
    if whole_num_list != []:
        amount += int(whole_num_list[0])
        
    # Add one if amount is detected
    if amount == 0:
        amount += 1
        
    # Set unit-value mappings
    unit_list = {
        'lb': 0.00220462,
        'bunch': 0.1,
        'liter': 0.236588,
        'quart': 0.25,
        'pinch': 0.355625,
        'pint': 0.5,
        'oz': 8,
        'ounce': 8,
        'thumb': 9.5,
        'tablespoon': 16,
        'tbsp': 16,
        'tsp': 48,
        'ml': 236.588
    }
    
    # Check for units
    for unit_name, unit_val in unit_list.items():
        if str_weight.find(unit_name) > -1:
            div_weight = unit_val
    
    # Compute weight
    weight = amount / div_weight
    
    return weight

def prepare_df(dish_dict):
    '''Create DataFrame out of dish dictionary'''
    df_ing = pd.DataFrame.from_dict(dish_dict).T
    df_ing = df_ing[df_ing.cuisine.str.contains('Filipino', na=False)]

    df_ing = df_ing.drop(columns=["course","cuisine","prep_time","cook_time","total_time","servings","calories"])
    df_ing = df_ing.sort_index(axis=1)
    df_ing = df_ing.dropna(axis=1, how='all')
    df_ing.fillna(0)
    
    cat_ing_dict = {
        'yeast': r'(yeast)',
        'wrapper': r'(wrapper)',
        'worcestershire_sauce': r'(worcestershire)',
        'winged_bean': r'(winged bean)',
        'vinegar': r'(vinegar)',
        'turmeric': r'(tumeric)',
        'tomato': r'(tomato)',
        'tofu': r'(tofu)',
        'toasted_rice_powder': r'(toasted rice powder)',
        'taro': r'(taro)',
        'sweet_potato': r'(sweet potato)',
        'sugar': r'(sugar)',
        'star_anise': r'(star anise)',
        'squash': r'(kalabasa|squash)',
        'soy_sauce': r'(soy sauce)',
        'sinigang_mix': r'(sinigang)',
        'shortening': r'(shortening)',
        'sesame_oil': r'(sesame oil)',
        'scallion': r'(scallion)',
        'sayote': r'(sayote)',
        'salted_egg': r'(salted egg)',
        'safflower_oil': r'(safflower oil)',
        'raisins': r'(raisins)',
        'radish': r'(radish)',
        'potato': r'(potato)',
        'pork_insides': r'(pig’s liver|pig’s heart|pig’s small intestine|bung|pig cheeks|pig heart|pig kidney|pig stomach|pork ears|pork large intenstine|pork liver|small intestine)',
        'pork_fat': r'(pork fat)',
        'pork_stock': r'(pork broth|pork stock|pork cube)',
        'pork_blood': r'(pork blood)',
        'pork_and_beans': r'(pork and beans)',
        'pie_crust': r'(pie)',
        'pickle': r'(pickle|relish)',
        'pepper_leaf': r'(pepper leaves)',
        'pechay': r'(pechay)',
        'peanut': r'(peanut)',
        'patola': r'(patola)',
        'parsley': r'(parsley)',
        'paprika': r'(paprika)',
        'papaya': r'(papaya)',
        'oyster_sauce': r'(oyster)',
        'onion': r'(onion)',
        'olive_oil': r'(olive oil)',
        'olive': r'(olive)',
        'okra': r'(okra)',
        'nutmeg': r'(nutmeg)',
        'noodle': r'(noodle|pancit|sotanghon|misua|miswa)',
        'mushroom': r'(mushroom)',
        'munggo': r'(mung)',
        'mirin': r'(mirin)',
        'mayonnaise': r'(mayonnaise)',
        'malunggay': r'(malunggay)',
        'liver_spread': r'(liver)',
        'lemongrass': r'(lemongrass)',
        'leeks': r'(leeks)',
        'lechon_sauce': r'(lechon)',
        'kasubha': r'(kasubha)',
        'kangkong': r'(spinach|kangkong)',
        'jicama': r'(jicama)',
        'jackfruit': r'(jackfruit)',
        'ice': r'(ice)',
        'hotdog': r'(hotdog)',
        'hot_sauce': r'(hot)',
        'honey': r'(honey)',
        'hoisin_sauce': r'(hoisin)',
        'green_pea': r'(green pea|pigeon pea)',
        'green_bean': r'(green beans|sitaw|snake beans|string beans|snap pea|snow pea)',
        'glutinous_rice': r'(glutinous rice)',
        'ginger': r'(ginger)',
        'ginataang_gulay_mix': r'(ginataang)',
        'garlic': r'(garlic)',
        'flour': r'(flour)',
        'eggplant': r'(eggplant|talong)',
        'egg': r'(egg)',
        'curry_powder': r'(curry)',
        'cucumber': r'(cucumber)',
        'cream': r'(cream)',
        'cooking_wine': r'(wine)',
        'cooking_oil': r'(cooking oil|vegetable oil)',
        'coconut_water': r'(coconut water)',
        'coconut_milk': r'(coconut cream|coconut milk)',
        'coconut_meat': r'(coconut meat)',
        'clear_soda': r'(7-up|sprite|clear softdrink)',
        'cinnamon': r'(cinnamon)',
        'sausage': r'(chinese sauage|chorizo)',
        'chicken_stock': r'(chicken broth|chicken cube)',
        'chicharon': r'(chicharon)',
        'cheese': r'(cheese)',
        'celery': r'(celery)',
        'carrot': r'(carrot)',
        'canned_meat': r'(potted meat|luncheon meat)',
        'calamansi': r'(calamansi|lemon|lime)',
        'cabbage': r'(cabbage)',
        'butter': r'(butter|margarine)',
        'broccoli': r'(broccoli)',
        'bread': r'(bread)',
        'bok_choy': r'(bok choy|bokchoy)',
        'black_soda': r'(coke|cola)',
        'black_bean': r'(black bean)',
        'beer': r'(beer)',
        'beef_insides': r'(lard|cow|beef heart|beef kidney|beef large instestine|beef liver|beef neck bone|beef small intestine|bile|tripe|tongue|tripe|lengua)',
        'beef_stock': r'(beef cube|beef bouillon|bulalo|beef broth|beef stock)',
        'bay_leaf': r'(bay)',
        'bamboo_shoots': r'(bamboo shoots)',
        'baking_powder': r'(baking powder)',
        'annatto': r'(annatto)',
        'ampalaya': r'(ampalaya)',
        'adobo_sauce': r'(adobo)',
        'achiote': r'(achiote)',
        'tomato_liquid': r'(ketchup|tomato sauce|tomato paste|spaghetti sauce)',
        'banana_flower': r'(blossom)',
        'pepper': r'(white pepper|black pepper|crushed pepper|peppercorn)',
        'chili': r'(chili|pepper flakes|serrano pepper|sili|jalapeno|ghost pepper|green pepper)',
        'bell_pepper': r'(bell pepper)',
        'bagoong': r'(alamang|shrimp paste|balaw)',
        'liquid_seasoning': r'(liquid seasoning|savorrich|marinade)',
        'chickpea': r'(chick pea|garbanzos)',
        'chicken_insides': r'(chicken gizzard|chicken hear|chicken liver)',
        'cornstarch': r'(cornstarch)',
        'corned_beef': r'(corned beef)',
        'fish_sauce': r'(fish sauce)',
        'pineapple_juice': r'(pineapple juice|juice from the canned tidbits)',
        'shrimp_cube': r'(shrimp cube)',
        'watermelon': r'(watermelon)',
        'milk': r'(milk)',
        'pea': r'(pea)',
        'pasta': r'(spaghetti|macaroni)',
        'shrimp': r'(shrimp)',
        'pineapple': r'(pineapple)',
        'water': r'(water)',
        'salt': r'(salt)',
        'rice': r'(rice|sinangag)',
        'pork': r'(pork|pig|lechon)',
        'corn': r'(corn)',
        'chicken': r'(chicken)',
        'beef': r'(beef|steak|oxtail|ox tail|sirloin|bistek)',
        'banana': r'(banana|plantain)'
    }

    # Create new DataFrame
    df_new = pd.DataFrame(index=df_ing.index)

    # Loop through the categories
    for cat_name, cat_regex in cat_ing_dict.items():
        # Look for columns that contains the query
        ing_filter = (
            df_ing.columns.to_series()
            .str.contains(cat_regex, case=False, regex=True)
        )

        # Filter columns that satisfy the query
        filter_cols = ing_filter[ing_filter].index
        
        print(f"Columns obtained for {cat_name}:", filter_cols)

        # Add columns to the new DataFrame
        df_new[cat_name] = df_ing[filter_cols].sum(axis=1)

        # Drop columns to prevent reusing of ingredients
        df_ing.drop(columns=filter_cols, inplace=True)
    
    df_new = df_new.rename_axis('dish_name').reset_index()
    return df_new

def export_sql(df_dish):
    """Export DataFrame to SQL file"""
    # Open connection to DB file
    conn = sqlite3.connect('ulam.db')

    # Transform DataFrame into DB table
    df_dish.to_sql('rekado', con=conn, if_exists='replace', index=False)
    
    # Close connection
    conn.close()


In [3]:
dish_urls = get_dish_urls()

Extraction of URLs done. Total number of dishes: 788


In [4]:
dish_dict = get_ingredients(dish_urls)

Getting details for:  Sinarsahang Manok
Getting details for:  Pinoy Chicken Curry Recipe
Getting details for:  Ketchup Fried Chicken
Getting details for:  Chicken Barbecue
Getting details for:  Father’s Day Chicken Macaroni Salad
Getting details for:  Chicken Afritada Recipe
Getting details for:  Chicken Paksiw Recipe
Getting details for:  Chicken Igado
Getting details for:  Sinigang Mix Inasal na Manok
Getting details for:  Chicken Binagoongan Recipe
Getting details for:  Sweet and Sour Chicken Meatballs
Getting details for:  Pineapple Chicken Afritada
Getting details for:  Making McDonalds Style Chicken Nuggets at home
Getting details for:  How to Make Crispy Fried Chicken (Spicy Version)
Getting details for:  How to Cook Chicken Kare-Kare sa Gata
Getting details for:  Spicy Sotanghon Chicken Soup
Getting details for:  Pininyahang Manok Recipe (Pineapple Chicken)
Getting details for:  Pininyahang Manok sa Gata (Pineapple Chicken in Coconut Milk)
Getting details for:  Chicken Feet in 

Getting details for:  Chicken with Ampalaya in Oyster Sauce
Getting details for:  Easy Chicken Curry
Getting details for:  Asian Fried Chicken
Getting details for:  Chicken and Ham Sopas Recipe
Getting details for:  Chicken Carbonara
Getting details for:  Barbecued Chicken Lettuce Wrap
Getting details for:  Chicken Siomai
Getting details for:  Sizzling Chicken Sisig
Getting details for:  Arroz Caldo with Tokwa (Chicken Congee with Fried Tofu)
Getting details for:  Fried Chicken Potato Salad
Getting details for:  Pesang Manok (Chicken Soup in Ginger Broth)
Getting details for:  Chicken Giniling with Green Peas
Getting details for:  Bean Sprout Chicken
Getting details for:  Asian Chicken Soup
Getting details for:  Asian Grilled Chicken and Corn
Getting details for:  Pulled Chicken BBQ Sandwich recipe
Getting details for:  Sautéed String Beans with Chicken
Getting details for:  Easy Chicken Tacos
Getting details for:  Sweet Chicken Spaghetti
Getting details for:  Chicken Piccata
Getting d

Getting details for:  Pork Afritada Recipe
Getting details for:  Pork Dinuguan Recipe
Getting details for:  Lumpia Recipe
Getting details for:  Sinigang
Getting details for:  Super Crispy Pata with Yummy Sawsawan
Getting details for:  How to Cook Embutido
Getting details for:  How to Cook Bicol Express
Getting details for:  Skinless Longganisa Recipe
Getting details for:  Spicy Pork Kaldereta
Getting details for:  Sisig
Getting details for:  How to Cook Lumpiang Shanghai
Getting details for:  How to Cook Pork Menudo – Filipino Recipe
Getting details for:  Filipino Style Spaghetti Recipe
Getting details for:  Pork Dinakdakan Recipe
Getting details for:  Pork Kare Kare Recipe
Getting details for:  Ginataang Lechon Kawali with Kalabasa and Long Green Beans
Getting details for:  Root Beer Pork Steak
Getting details for:  Sizzling Liempo Sisig Recipe
Getting details for:  Filipino Fried Pork Chop Recipe with Tomato and Salted Egg
Getting details for:  Pocherong Baboy Recipe
Getting details 

Getting details for:  Ginataang Langka with Pork Recipe
Getting details for:  Pork Burger Patties
Getting details for:  Crab and Pork Shumai Recipe (Siomai)
Getting details for:  Adobo Pork Chops
Getting details for:  Simple Baked Pork Chop Recipe
Getting details for:  Pork Menudo with Raisins and Green Peas
Getting details for:  Sinigang na Liempo sa Sampaloc with Gabi
Getting details for:  Easy Pork Afritada Recipe
Getting details for:  Pamplina Recipe (Pork and Chorizo Soup with Beans)
Getting details for:  Linat an Recipe (Boiled Pork Soup with Lemongrass)
Getting details for:  Inihaw na Pork Chop
Getting details for:  Nilagang Baboy (Boiled Pork Soup)
Getting details for:  Bas uy Recipe (Pork and Liver Soup with Lemongrass and Chayote)
Getting details for:  Crispy Oven Broiled Liempo
Getting details for:  Adobong Bulalo (Beef Shank Adobo)
Getting details for:  Deep Fried Seasoned Pork Chop Recipe
Getting details for:  Pancit Bato Guisado
Getting details for:  Apple Pork Pot Roast


Getting details for:  Beef Pot Roast
Getting details for:  Gotong Batangas Recipe
Getting details for:  Sous Vide Strip Steak Recipe
Getting details for:  Beef Short Rib Kare Kare
Getting details for:  Beef Short Rib Sinigang Na May Pakwan
Getting details for:  Corned Beef with Cabbage and Potato
Getting details for:  Bulalo Beef Pares Recipe
Getting details for:  Stir Fried Pepper Steak
Getting details for:  Batangas Bulalo Recipe
Getting details for:  Sinabawang Corned Beef Recipe
Getting details for:  Stir Fried Beef with Oyster Sauce and Snap Peas
Getting details for:  Lengua in White Mushroom Sauce
Getting details for:  Filipino Beef Morcon with Garlic Fried Rice and Eggs
Getting details for:  Lengua Estofado Recipe
Getting details for:  How To Cook Beef Kaldereta
Getting details for:  Slow Cooked Beefsteak (Bistek Tagalog)
Getting details for:  Beef Pares Mami Noodles Recipe
Getting details for:  Cheesy Baked Macaroni Recipe
Getting details for:  Classic Beef Morcon Recipe
Gettin

In [5]:
df_dish = prepare_df(dish_dict)
df_dish

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Columns obtained for yeast: Index(['dry yeast'], dtype='object')
Columns obtained for wrapper: Index(['lumpia wrapper', 'wonton wrapper'], dtype='object')
Columns obtained for worcestershire_sauce: Index(['Worcestershire Sauce', 'Worcestershire sauce'], dtype='object')
Columns obtained for winged_bean: Index(['winged bean'], dtype='object')
Columns obtained for vinegar: Index(['Spiced Vinegar', 'coconut vinegar', 'red wine vinegar or cane vinegar',
       'vinegar', 'white or apple cider vinegar', 'white or cane vinegar',
       'white vinegar'],
      dtype='object')
Columns obtained for turmeric: Index([], dtype='object')
Columns obtained for tomato: Index(['14.5 oz. can diced tomato', '6 oz can tomato paste',
       '8 oz. can tomato sauce', '8oz. can tomato sauce',
       'Filipino style tomato sauce', 'diced tomato', 'large plum tomato',
       'medium plum tomato', 'medium ripe tomato', 'medium tomato',
       'medium tomatoes', 'plum tomato', 'ripe tomato',
       'small can tom

Columns obtained for pepper: Index(['Fish sauce and crushed peppercorn to taste',
       'Fish sauce and ground black pepper to taste',
       'Fish sauce and ground white pepper to taste', 'Ground black pepper',
       'Ground black pepper and shrimp paste', 'Ground black pepper to taste',
       'Patis and ground black pepper',
       'Patis and ground black pepper to taste',
       'Salt and ground black pepper to taste',
       'Salt and ground white pepper to taste', 'crushed black pepper',
       'crushed pepper', 'crushed peppercorn', 'crushed whole peppercorn',
       'fish sauce and ground black pepper to taste', 'ground black pepper',
       'ground black pepper to taste', 'ground white pepper',
       'ground white pepper to taste', 'peppercorn', 'whole peppercorn',
       '• A dash of ground black pepper'],
      dtype='object')
Columns obtained for chili: Index(['Jalapeno peppers', 'Serrano pepper', 'Serrano peppers', 'Thai chili',
       'Thai chili or siling labuyo', 'Th

,dish_name,yeast,wrapper,worcestershire_sauce,winged_bean,vinegar,turmeric,tomato,tofu,toasted_rice_powder,...,shrimp,pineapple,water,salt,rice,pork,corn,chicken,beef,banana
0,Sinarsahang Manok,0.0,0.0,0.0,0.0,0.0000,0.0,2.5,0.0,0.0,...,0.0,0.0,1.50,0.000000,0.0,0.0,0.000,680.389364,0.000000,0.0
1,Pinoy Chicken Curry Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,1.00,0.000000,0.0,0.0,0.000,907.185819,0.000000,0.0
2,Ketchup Fried Chicken,0.0,0.0,0.0,0.0,0.3125,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.000000,0.0,0.0,0.000,3.000000,0.000000,0.0
3,Chicken Barbecue,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,2.000000,0.0,0.0,0.000,4.000000,0.000000,0.0
4,Chicken Paksiw Recipe,0.0,0.0,0.0,0.0,0.5000,0.0,0.0,0.0,0.0,...,0.0,0.0,1.00,1.000000,0.0,0.0,0.000,453.592909,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Papaitan Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,8.00,0.156250,0.0,0.0,0.000,0.000000,226.796455,0.0
278,Morcon Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.005208,0.0,0.0,0.000,0.000000,907.185819,0.0
279,Beef Mechado Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,1.0,0.0,0.0,...,0.0,0.0,1.00,1.000000,0.0,0.0,0.000,0.000000,907.185819,0.0
280,Bulalo Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,4.25,0.000000,0.0,0.0,1.125,0.000000,907.185819,0.0


In [6]:
export_sql(df_dish)